In [1]:
%%sh
pwd

/home/lausena/developer/repos/spacecraft-pose-pose-estimation-runtime/juptyer-notebooks


In [2]:
!pip install transforms3d --quiet

In [10]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import transforms3d.quaternions as quat
from scipy import signal
import scipy.fft as sfft

In [11]:
camera_matrix = np.array([[5.2125371e+03, 0.0000000e+00, 6.4000000e+02],
                          [0.0000000e+00, 6.2550444e+03, 5.1200000e+02],
                          [0.0000000e+00, 0.0000000e+00, 1.0000000e+00]])

# range 01f0f7459b,0,299.2
#01f0f7459b,1,

In [12]:
root = '/home/lausena/developer/repos/spacecraft-pose-pose-estimation-runtime/'

In [85]:
# image 001:
# trans 7.64022827,-46.98920059,-42.21747971
# quat ,0.98997474,-0.0941886,-0.07856753,0.07003857
orig_trans = np.array([7.64022827,-46.98920059,-42.21747971])
orig_quat = np.array([0.98997474,-0.0941886,-0.07856753,0.07003857])

In [121]:
def execute_test(img1, img2, orig_trans, orig_quat):
    orb = cv2.ORB_create()
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)
    # Match keypoints
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    # Extract matched keypoints
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    # Estimate homography
    # H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC)
    H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC)

    num, Rs, Ts, Ns = cv2.decomposeHomographyMat(H, camera_matrix)
    a = Rotation.from_matrix(Rs[0]).as_quat()
    b = Rotation.from_matrix(H).as_quat()

    # print('Quaternion test:')
    # print(f'{mean_absolute_error(a, orig_quat)} : Decomposition')
    # print(mean_absolute_error(b, orig_quat))
    # print()
    print('Translation test:')
    print(f'{mean_absolute_error(np.array([0.,0.,0.]), orig_trans)} Base case')
    print(mean_absolute_error(Ts[0], orig_trans))
    print()

In [122]:
for i in range(1, 11):
    print(f'Image = {i}')
    img1path = os.path.join(root, 'data/images/01f0f7459b/000.png')
    img2path = os.path.join(root, f'data/images/01f0f7459b/00{i}.png')
    
    img1 = cv2.imread(img1path, cv.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img2path, cv.IMREAD_GRAYSCALE)
    img1full = cv2.imread(img1path, cv.IMREAD_GRAYSCALE)
    img2full = cv2.imread(img2path, cv.IMREAD_GRAYSCALE)

    labels = np.array([[7.64022827,-46.98920059,-42.21747971,0.98997474,-0.0941886,-0.07856753,0.07003857],
        [12.37142944,-36.08272934,-76.19368744,0.98547542,-0.09323603,-0.13402745,0.04670797],
        [48.83045959,-62.45590591,-149.01345825,0.9463315,-0.15760925,-0.27446723,0.06544811],
        [42.9213562,3.1468811,-156.43341064,0.96294099,-0.05801675,-0.26296937,-0.0150205],
        [25.3291626,-15.59156799,-118.26628113,0.97709697,-0.05717814,-0.20439324,0.01534067],
        [55.51976013,-30.00940132,-172.91856384,0.94631386,-0.10770535,-0.30420068,0.01874543],
        [ 69.65390015,-14.23764992,-194.42814636,0.93474096,-0.0921587,-0.34309945,-0.00697307],
        [67.03590393,57.95062637,-181.86766052,0.94076264,0.03300816,-0.32461578,-0.09219801],
        [95.08578491,70.74472046,-209.08718872,0.91551983,0.02235752,-0.38302255,-0.12090118],
        [115.60142517,80.90587616,-223.3885498,0.89717531,0.01968047,-0.41800421,-0.14128494]])

    original_trans = labels[i-1][:3]
    original_quat = labels[i-1][3:]
    
    execute_test(img1, img2, original_trans, original_quat)

Image = 1
Translation test:
32.28230285666667 Base case
33.59082343862354

Image = 2
Translation test:
41.54928207333334 Base case
45.747369978317465

Image = 3
Translation test:
86.76660791666666 Base case
91.3611142404352

Image = 4
Translation test:
67.50054931333334 Base case
53.65835905079221

Image = 5
Translation test:
53.06233724 Base case
57.6353925224469

Image = 6
Translation test:
86.14924176333334 Base case
39.95098152651341

Image = 7
Translation test:
92.77323214333335 Base case
39.434431765392404

Image = 8
Translation test:
102.28473027333332 Base case
57.15854380839394

Image = 9
Translation test:
124.97256469666667 Base case
89.59045605843141

Image = 10


[ WARN:0@2219.469] global loadsave.cpp:248 findDecoder imread_('/home/lausena/developer/repos/spacecraft-pose-pose-estimation-runtime/data/images/01f0f7459b/0010.png'): can't open/read file: check file path/integrity
[ WARN:0@2219.497] global loadsave.cpp:248 findDecoder imread_('/home/lausena/developer/repos/spacecraft-pose-pose-estimation-runtime/data/images/01f0f7459b/0010.png'): can't open/read file: check file path/integrity


error: OpenCV(4.9.0) /home/conda/feedstock_root/build_artifacts/libopencv_1704864625327/work/modules/core/src/batch_distance.cpp:274: error: (-215:Assertion failed) type == src2.type() && src1.cols == src2.cols && (type == CV_32F || type == CV_8U) in function 'batchDistance'


In [73]:
sift = cv.SIFT_create()

In [74]:
keypoints1, descriptor1 = sift.detectAndCompute(img1, None)
keypoints2, descriptor2 = sift.detectAndCompute(img2, None)

In [75]:
FLANN_INDEX_KDTREE = 1
indexParams = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
searchParams = dict(checks=50)
flann = cv.FlannBasedMatcher(indexParams, searchParams)
nNeighbors = 2

In [76]:
matches = flann.knnMatch(descriptor1, descriptor2, k=nNeighbors)

In [77]:
goodMatches = []
for m,n in matches:
    if m.distance < 0.7*n.distance:
        goodMatches.append(m)

minGoodMatches = 5

In [78]:
len(goodMatches)

2

In [79]:
if len(goodMatches) > minGoodMatches:
    source_points = np.float32([keypoints1[m.queryIdx].pt for m in goodMatches]).reshape(-1,1,2)
    destination_points = np.float32([keypoints2[m.queryIdx].pt for m in goodMatches]).reshape(-1,1,2)
    errorThreshold = 5
    M, mask = cv.findHomography(source_points, destination_points, cv.RANSAC, errorThreshold)
    matches_mask = mask.ravel().tolist()
    h,w = img1.shape
    img_border = np.float32([[0,0], [0, h-1], [w-1, h-1], [w-1,0]]).reshape(-1,1,2)
    warped_img_border = cv.perspectiveTransform(img_border, M)
    img2_poly = cv.polylines(img2, [np.int32(warped_img_border)], True, 255, 3, cv.LINE_AA)
else:
    print('Not enough good matches')

Not enough good matches


In [80]:
num, Rs, Ts, Ns = cv2.decomposeHomographyMat(M, camera_matrix)

In [81]:
a = Rotation.from_matrix(Rs[0]).as_quat()
b = Rotation.from_matrix(H).as_quat()

In [82]:
print(mean_absolute_error(a, orig_quat))
print(mean_absolute_error(b, orig_quat))

0.5360092775186155
0.30566994567587386


In [83]:
print(mean_absolute_error(np.array([0.,0.,0.]), orig_trans))
print(mean_absolute_error(Ts[0], orig_trans))

32.28230285666667
1088.50286222482
